In [1]:
import numpy as np
import tensorflow as tf
import random

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    path = 'imdb.npz',
    num_words = None,
    skip_top = 0,
    maxlen = None,
    seed = 113,
    start_char = 1,
    oov_char = 2,
    index_from = 3)

positive_reviews = []
negative_reviews = []
for i in range(0, len(x_train)):
    if y_train[i] == 1:
        positive_reviews.append(x_train[i])
    else:
        negative_reviews.append(x_train[i])

C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [19]:
selected_positive_reviews = random.sample(positive_reviews, np.int(len(positive_reviews) / 2))
balanced_positive_reviews = np.tile(selected_positive_reviews, 2)
word_index = tf.keras.datasets.imdb.get_word_index()

False

In [20]:
x_array = []
y_array = [1] * len(balanced_positive_reviews) +  [0] * len(negative_reviews)
for i in range(0, len(balanced_positive_reviews)):
    x_array.append(balanced_positive_reviews[i])
for i in range(0, len(negative_reviews)):
    x_array.append(negative_reviews[i])
train_data = tf.keras.preprocessing.sequence.pad_sequences(x_array,padding='post', maxlen=256)
test_data = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=256, padding='post')
print(train_data.shape, test_data.shape)

(25000, 256) (25000, 256)


In [22]:
from tensorflow.python import keras
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(88587, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          5669568   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               197632    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
______________________________________________

In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
from sklearn.model_selection import train_test_split

partial_x_train, x_val, partial_y_train, y_val = train_test_split(train_data, y_array, test_size=0.33, random_state=42)

In [30]:
model.fit(partial_x_train, partial_y_train, epochs=4, batch_size=512, validation_data=(x_val, y_val), verbose=1)

Train on 16750 samples, validate on 8250 samples
Epoch 1/4
16750/16750 [==============================] - 653s 39ms/sample - loss: 0.6757 - acc: 0.5981 - val_loss: 0.5544 - val_acc: 0.7732
Epoch 2/4
16750/16750 [==============================] - 695s 41ms/sample - loss: 0.3937 - acc: 0.8390 - val_loss: 0.2792 - val_acc: 0.8914
Epoch 3/4
16750/16750 [==============================] - 1082s 65ms/sample - loss: 0.1368 - acc: 0.9539 - val_loss: 0.2498 - val_acc: 0.9141
Epoch 4/4
16750/16750 [==============================] - 1501s 90ms/sample - loss: 0.0500 - acc: 0.9861 - val_loss: 0.2422 - val_acc: 0.9168


In [31]:
predictions = []
predictions_probabilities = model.predict(train_data)

NameError: name 'aggregated' is not defined

In [32]:
for i in range(0, len(predictions_probabilities)):
    if predictions_probabilities[i] <= 0.5:
        predictions.append([0])
    else:
        predictions.append([1])

In [54]:
import pandas as pd
y_label = np.array(y_array)
flat_list = [item for sublist in predictions[0] for item in sublist]
np.array(flat_list)
predictions = np.array(flat_list)

TypeError: 'numpy.int32' object is not iterable

In [68]:
pd.crosstab(y_label, np.array(flat_list), rownames=['Actual'], colnames=['Predicted'])

Predicted,0,1
Actual,,
0,12010,490
1,258,12242


In [70]:
from sklearn.metrics import classification_report

report = classification_report(y_label,flat_list)

In [71]:
report

'              precision    recall  f1-score   support\n\n           0       0.98      0.96      0.97     12500\n           1       0.96      0.98      0.97     12500\n\n    accuracy                           0.97     25000\n   macro avg       0.97      0.97      0.97     25000\nweighted avg       0.97      0.97      0.97     25000\n'